## Japan Public Company Financial Information Crawler
##### Last Update: 2022-08-03
#### Author: Wonho Lim 
#### E-mail: wonholim02@gmail.com
#### Python Version: Python 3.10.4 (ipykernel)
#### Chrome Version: Chrome 103.0.5060.134 (64-bit)
#### Chrome Driver Version: ChromeDriver 103.0.5060.134

#### Crawled Website
Yahoo Finance MAIN: https://finance.yahoo.com/

Description: This is public company financial statement web crawling code for Yahoo finance. This crawler focuses on getting Japanese public company data. Including Japan, yahoo finance have various general/financial information of companies around the world, and has reasonable amount of information. But their server status is the worst so far, and error occurs a lot of time. For crawling, as long as the website has not been modified, the code below must be run properly if and only if it is run in an order. Moreover, environment setting must be fixed based on user's computer/server setting and location. However, one need to utilize a lot of crawlers at the same time so that those companies that are not crawled due to loading status could be searched again by another crawler. Also, full excel list of name of the companies are needed to be collected beforehead as long as this crawler uses serach engine.

### Method
#### - Because of the special structure, entirely new method is used for yahoo finance crawling.
#### 1.Get complete list (xlsx) of Japanese public company names
#### 2.Extract each company name from the file
#### 3.Search on yahoo finance engine using chromedriver
#### 4.Get data that we need
#### 5.Store as list, and write on new csv file

### Tips
#### Yahoo Finance has unstable server
##### - Error occurs randomly based on loading time
#####  Some of them cannot be solved by time sleep
#### For Better Accuracy, we need to run crawlers as many as possible

### 1. Importing useful open source librabries - utilized BS4 and Selenium Web Driver for crawling

In [2]:
import bs4
import time
import csv 
import pandas as pd 
from platform import python_version
import requests
import lxml 
import xlrd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By

### 2. Environment
#### - Chromdriver must be located in local/cloud PC folder.
#### - One can download proper version at: chromedriver.chromium.org/downloads
#### - Path should be modified based on local environment

In [2]:
print(python_version())
print(pd.__version__)

3.9.12
1.4.2


### 3. File Setting
#### - CSV file name and Encoding must be checked based on user preference.
#### - CSV file will be newly added on the folder where ipynb file is located.
#### - Column can be changed if it is required, but the main crawler code must be also modified for accuracy.
#### - DO NOT RUN THIS CODE AGAIN AFTER CRAWLING OR THE BLANK FILE WILL REPLACE CURRENT FILE.

In [3]:
"""
cover = open('Japan_2022_Yahoo_2022-06-21_Chris - 1.csv','w', newline='', encoding='CP949')
path = r'C:\Users\user\Desktop\자료\chromedriver'
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드','현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])
"""

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 104-105: truncated \UXXXXXXXX escape (3254100117.py, line 6)

### 4. File Importing with Pandas
#### - Using Pandas, read the list of company names
#### - This crawler utilizes search engine service to get data
#### - Test if the excel file is in right format

In [3]:
df = pd.read_excel (r'C:\Users\user\Desktop\Japan\Japan Company List - Completed.xlsx')

In [7]:
print(df['현지언어기업명'].iloc[600])
print(df['기업상태'].iloc[500])

TOYOTA BOSHOKU CORPORATION
Prime


In [5]:
for i in range (0,650):
    print(df['현지언어기업명'].iloc[i])

KYOKUYO CO.,LTD.
Nippon Suisan Kaisha,Ltd.
Maruha Nichiro Corporation
YUKIGUNI MAITAKE CO.,LTD.
KANEKO SEEDS CO.,LTD.
SAKATA SEED CORPORATION
HOKUTO CORPORATION
AKIKAWA FOODS & FARMS CO.,LTD.
AXYZ CO.,Ltd.
HOB Co.,Ltd.
Berg Earth co.,ltd.
Hokuryo Co.,Ltd.
RUDEN HOLDINGS CO.,Ltd.
mbs,inc.
West Holdings Corporation
SHO-BOND Holdings Co.,Ltd.
MIRAIT Holdings Corporation
INTERLIFE HOLDINGS CO.,LTD.
Tama Home Co.,Ltd.
Sanyo Homes Corporation
Nippon Aqua Co.,Ltd.
First-corporation Inc.
Lib Work Co.,Ltd.
DORYOKU Co.,Ltd.
BESTERRA CO.,LTD
JESCO Holdings,Inc.
Robot Home,Inc.
Fit Corporation
YASUE CORPORATION
YAMAZEN HOMES CO.,LTD.
Giken Holdings Co.,Ltd.
Hikari Holdings Co.,Ltd.
CANDEAL Co.,Ltd.
ITbook Holdings Co.,LTD.
TANAKEN
KHC Ltd.
YOKOHAMA WRIGHT Industries Co.,LTD
Chugai Mining Co.,Ltd.
Sumiseki Holdings,Inc.
Nittetsu Mining Co.,Ltd.
MITSUI MATSUSHIMA HOLDINGS CO.,LTD.
INPEX CORPORATION
Japan Petroleum Exploration Co.,Ltd.
K&O Energy Group Inc.
SDS HOLDINGS Co.,Ltd.
Daiseki Eco.Solution 

### 4. Main Crawler
#### - Uncollectable or unavailable data was collected as "" for convenience.
#### - Data Cleansing might be required after crawling sometiems, but it will not be tough as one knows how to use excel.
#### - Some directions and explanations are written as comment below.
#### - The order of append must be depend on the order of column name assigned above.
#### - Lots of try-except function is used to avoid error caused by non-existing pages or information.
#### - range must be modified by each crawler.

### - Condensed version only collects financial information.
### - Non-condensed version causes some loading problem, so I highly recommend to use condensed version.

### 2022

In [4]:
df = pd.read_excel (r'C:\Users\user\Desktop\Japan\Japan Company List - Completed.xlsx')
path = r'C:\Users\user\Desktop\자료\chromedriver'
cover = open('Japan2022.csv','w', newline='', encoding='UTF8')
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드','현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])
# Basic Setting
failList = []
driver = webdriver.Chrome(path)
driver.get('https://finance.yahoo.com/quote/1301.T/holders?p=1301.T')
time.sleep(3)
# Range must be changed
for i in range (0,3819):
    try:
        # Search Setting
        time.sleep(1)  
        name = df['현지언어기업명'].iloc[i]
        info = []
        # Status Check
        print(i)
        print("out of")
        print(3820)
        print(name)

        # Enter name of company in searchbox, and wait for 2 seconds.
        try:
            driver.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div[1]/div/div[2]/div/form/input[1]").send_keys(df['현지언어기업명'].iloc[i])
        except:
            driver.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div/div/div[1]/div/div[2]/div/form/input[1]").send_keys(df['현지언어기업명'].iloc[i])
        time.sleep(3)
        # Get to the company information page
        try:
            driver.find_element(By.XPATH, value='//*[@id="result-quotes-0"]/div[1]').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[1]/div/div/div/div/div/div/div/div/div/div/div/div/div[2]/div/div[2]/form/div/div[1]/div/ul[1]/li[1]').click()
        time.sleep(4)
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/div/div/section/div/ul/li[6]/a').click()
        time.sleep(4)

        #Country Information
        info.append("JPN") 
        info.append("Nihon") 
        info.append("Japan") 
        info.append("UTC+09:00") 
        info.append("아시아") 
        info.append("6100000000000 USD") 
        info.append("125502000") 
        info.append("동북아시아") 

        #Company Identification
        info.append("JPN202206" + name)
        info.append(name)
        info.append(name)

        #Descripton, Contact, Address, Extra Information, Management Information
        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan, Asia"
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan, Asia"
            except:
                try:
                    address1 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[1]').text + ", Japan"
                    address2 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[1]').text + ", Japan, Asia"
                except:
                    address1 = "Japan"
                    address2 = "Japan, Asia"
        try:
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[2]/p').text) 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[2]/p').text) 
        except:
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group). Located at " + address2 + ", they are providing services related to  " + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[4]').text + ". You can find more information on company's website, " + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[2]').text)
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group). Located at " + address2 + ", they are providing services related to  " + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[4]').text + ". You can find more information on company's website, " + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[2]').text)
        info.append("")
        info.append("")
        info.append("")

        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[1]').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[2]').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul').text) 
            except:
                info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(address1)
        info.append(address1)
        info.append(address2)
        info.append(address2)
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(df['기업상태'].iloc[i]) 
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[2]/span').text)
        except:
            info.append("Contact")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[2]/span').text)
        except:
            info.append("Contact")
        try:
            info.append("Executive Board")
        except:
            info.append("Contact")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[1]').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[1]').text) 
        except:
            info.append("")

        # Financial Information
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul/li[7]/a').click()
        time.sleep(4)
        #회계연도

        try:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[2]/a/div').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[2]/a/div/span').click()
        time.sleep(2)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button').click()
        except:
            time.sleep(2)
            try: 
                driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/svg').click()
            except:
                try:
                    time.sleep(2)
                    driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/svg/path').click()
                except:
                    driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/div').click()
        time.sleep(2)

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[1]/div[2]/span').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[1]/div/div[2]/span').text) 
            except:
                info.append("")

        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[2]/div[1]/div[1]/div[2]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[2]/div[1]/div[1]/div[2]/span').text  + "K Yen")
            except:
                info.append("")
        #비유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[2]/div[2]/div[1]/div[2]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[2]/div[2]/div[1]/div[2]/span').text  + "K Yen")
            except:
                info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[1]/div[2]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[1]/div[2]/span').text  + "K Yen")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[2]/div[1]/div[1]/div[2]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[2]/div[1]/div[1]/div[2]/span').text  + "K Yen")
            except:
                info.append("")
        #비유동부채금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[2]/div[2]/div[1]/div[2]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[2]/div[2]/div[1]/div[2]/span').text + "K Yen")
            except:
                info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[1]/div[2]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[1]/div[2]/span').text + "K Yen")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[2]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[2]/span').text + "K Yen")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[3]/div[1]/div[2]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[3]/div[1]/div[2]/span').text  + "K Yen")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[1]/a').click()
        time.sleep(4)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[2]/div[1]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[5]/div[1]/div[3]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[5]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[6]/div[2]/div[1]/div[1]/div[3]/span').text + "K Yen")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[6]/div[2]/div[1]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #금융비용금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[6]/div[2]/div[2]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[6]/div[2]/div[2]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #기타영업외수익금액
        info.append("")
        #기타영업외비용금액
        info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[7]/div[1]/div[3]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[23]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[8]/div[1]/div[3]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[8]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[9]/div[2]/div[1]/div[2]/div[1]/div[2]/div/div[1]/div[3]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[27]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[3]/a').click()
        time.sleep(4)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[3]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[3]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[2]/div[3]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[2]/div[2]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #재무정보중 계산예정
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")


        # Industry
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul/li[6]/a').click()
        time.sleep(4)
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[2]').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[2]').text)
        except:
            info.append("General Business")
            info.append("General Business")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[4]').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[4]').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[2]').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[2]').text)
            except:
                info.append("Product and Services")
                info.append("Product and Services")


        # Language Information
        info.append("JPN")
        info.append("Nihongo")
        info.append("Japanese")

        # Stock Market Information
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul/li[1]/a').click()
        time.sleep(4)
        info.append("JPX")
        info.append("Japan Exchange Group")
        info.append("Japan Exchange Group")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[6]/div/div[2]/div[1]/div[1]/h1').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[6]/div/div[2]/div[1]/div[1]').text)
            except:
                info.append("")
        #상장일자
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/div/div[2]/div[1]/table/tbody/tr[5]/td[2]').text + " JPY")
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/div/div[2]/div[1]/table/tbody/tr[6]/td[2]').text + " JPY")
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        #거래량
        info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/div/div[2]/div[1]/table/tbody/tr[7]/td[2]/fin-streamer').text)
        #시가총액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/div/div[2]/div[2]/table/tbody/tr[1]/td[2]').text)
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Japan")
        info.append("Japan, Asia")
        info.append("")
        info.append("")

        # Event
        info.append("Not Available")

        # Currency Information
        info.append("JPY")
        info.append("Yen")

        # Management
        info.append("Chris")
        info.append("Yahoo Finance")
        info.append("2022-06-21")

        # Status Management
        print(info)
        writing.writerow(info)
    except:
        try:
            info = []
            name = df['현지언어기업명'].iloc[i]
            #국가코드
            info.append("JPN")
            info.append("Nihon")
            info.append("Japan")
            info.append("UTC+09:00")
            info.append("아시아")
            info.append("6100000000000 USD")
            info.append("125502000")
            info.append("동북아시아")
            info.append("JPN202206" + name)
            info.append(name)
            info.append(name)
            address1 = "Japan"
            address2 = "Japan, Asia"
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group). Located at " + address2 + ", they are operated as public company. You can find more information on company's website.")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group). Located at " + address2 + ", they are operated as public company. You can find more information on company's website.")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append(address1)
            info.append(address1)
            info.append(address2)
            info.append(address2)
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append(df['기업상태'].iloc[i]) 
            info.append("")
            info.append("")
            info.append("Contact")
            info.append("Contact")
            info.append("Contact")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("TTM")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("General Business")
            info.append("General Business")
            info.append("Product and Services")
            info.append("Product and Services")
            info.append("JPN")
            info.append("Nihongo")
            info.append("Japanese")
            info.append("JPX")
            info.append("Japan Exchange Group")
            info.append("Japan Exchange Group")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Japan")
            info.append("Japan, Asia")
            info.append("")
            info.append("")
            info.append("Not Available")
            info.append("JPY")
            info.append("Yen")
            info.append("Chris")
            info.append("Yahoo Finance")
            info.append("2022-06-21")

            writing.writerow(info)
        except:
            failList.append(df['현지언어기업명'].iloc[i])
            
# Check failed result 
# - There are many ways to deal with failList: retry, ignore, manually add, find why error has occured.
# - This will be up to user's choice.
print("failLength:")
print(len(failList))
print(failList)

C:\Users\user\AppData\Local\Temp\ipykernel_3416\3694090256.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


1285
out of
3820
ONE CAREER Inc.


C:\Users\user\AppData\Local\Temp\ipykernel_3416\3694090256.py:26: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div[1]/div/div[2]/div/form/input[1]").send_keys(df['현지언어기업명'].iloc[i])
C:\Users\user\AppData\Local\Temp\ipykernel_3416\3694090256.py:28: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div/div/div[1]/div/div[2]/div/form/input[1]").send_keys(df['현지언어기업명'].iloc[i])


['JPN', 'Nihon', 'Japan', 'UTC+09:00', '아시아', '6100000000000 USD', '125502000', '동북아시아', 'JPN202206ONE CAREER Inc.', 'ONE CAREER Inc.', 'ONE CAREER Inc.', 'ONE CAREER Inc.(English: ONE CAREER Inc.) is a public company that is listed on JPX(Japan Exchange Group).', 'ONE CAREER Inc.(English: ONE CAREER Inc.) is a public company that is listed on JPX(Japan Exchange Group).', 'ONE CAREER Inc. provides recruitment DX support services through career data platform. It operates ONE CAREER and ONE CAREER PLUS job hunting media. The company was founded in 2015 and is based in Tokyo, Japan.', 'ONE CAREER Inc. provides recruitment DX support services through career data platform. It operates ONE CAREER and ONE CAREER PLUS job hunting media. The company was founded in 2015 and is based in Tokyo, Japan.', '', '', '', '81 3 6416 4088', '', '', 'https://onecareer.co.jp', '', '', '', '', '', 'Japan', 'Japan', 'Japan, Asia', 'Japan, Asia', '', '', '', '', 'Growth', 'Mr. Takashi Miyashita', 'Mr. Takashi 

1289
out of
3820
BPLATS,Inc.
['JPN', 'Nihon', 'Japan', 'UTC+09:00', '아시아', '6100000000000 USD', '125502000', '동북아시아', 'JPN202206BPLATS,Inc.', 'BPLATS,Inc.', 'BPLATS,Inc.', 'BPLATS,Inc.(English: BPLATS,Inc.) is a public company that is listed on JPX(Japan Exchange Group).', 'BPLATS,Inc.(English: BPLATS,Inc.) is a public company that is listed on JPX(Japan Exchange Group).', 'BPLATS,Inc. provides subscription services in Japan and internationally. The company offers Bplats platform, which provides an ecosystem that connects suppliers/sales destinations for contracts, transactions, and billing that supports the entire subscription for major companies. It also provides publishing and consulting services. The company was incorporated in 2006 and is headquartered in Tokyo, Japan.', 'BPLATS,Inc. provides subscription services in Japan and internationally. The company offers Bplats platform, which provides an ecosystem that connects suppliers/sales destinations for contracts, transactions, and

...

2806
out of
3820
HASHIMOTO SOGYO HOLDINGS CO.,LTD.
2807
out of
3820
YAMANO HOLDINGS CORPORATION
2808
out of
3820
Japan Lifeline Co.,Ltd.
2809
out of
3820
NICHIRYOKU CO.,LTD.
2810
out of
3820
SAIZERIYA CO.,LTD.
2811
out of
3820
KAN-NANMARU CORPORATION
2812
out of
3820
TAKASHO CO.,LTD.
2813
out of
3820
VT HOLDINGS CO.,LTD.
2814
out of
3820
ARGO GRAPHICS Inc.
2815
out of
3820
UORIKI CO.,LTD.
2816
out of
3820
IDOM Inc.
2817
out of
3820
Japan Medical Dynamic Marketing,INC.
2818
out of
3820
POPLAR CO.,Ltd.
2819
out of
3820
Carchs Holdings Co.,Ltd.
2820
out of
3820
MAC HOUSE CO.,LTD.
2821
out of
3820
UMENOHANA CO.,LTD.
2822
out of
3820
FUJI CORPORATION
2823
out of
3820
UNITED ARROWS LTD.
2824
out of
3820
Shinwa Co.,Ltd.
2825
out of
3820
SK JAPAN CO.,LTD.
2826
out of
3820
Daitron Co.,Ltd.
2827
out of
3820
TAY TWO CO.,LTD.
2828
out of
3820
HIDAY HIDAKA Corp.
2829
out of
3820
SIIX CORPORATION
2830
out of
3820
OM2Network Co.,Ltd.
2831
out of
3820
YU-WA Creation Holdings Co.,Ltd.
2832
out of
3820


3026
out of
3820
KOMATSU WALL INDUSTRY CO.,LTD.
3027
out of
3820
YAMAHA CORPORATION
3028
out of
3820
Kawai Musical Instruments Manufacturing Co.,Ltd.
3029
out of
3820
KIKUSUI CHEMICAL INDUSTRIES CO.,LTD.
3030
out of
3820
Cleanup Corporation
3031
out of
3820
PIGEON CORPORATION
3032
out of
3820
FUJICOPIAN CO.,LTD.
3033
out of
3820
TENMA CORPORATION
3034
out of
3820
KANEMATSU SUSTECH CORPORATION
3035
out of
3820
KING JIM CO.,LTD.
3036
out of
3820
KOKEN LTD.
3037
out of
3820
Zojirushi Corporation
3038
out of
3820
LINTEC Corporation
3039
out of
3820
Shin-Etsu Polymer Co.,Ltd.
3040
out of
3820
TOLI Corporation
3041
out of
3820
ITOKI CORPORATION
3042
out of
3820
Nintendo Co.,Ltd.
3043
out of
3820
LIHIT LAB.,INC.
3044
out of
3820
MITSUBISHI PENCIL COMPANY,LIMITED
3045
out of
3820
SHOFU INC.
3046
out of
3820
SHIGEMATSU WORKS CO.,LTD.
3047
out of
3820
TAKARA STANDARD CO.,LTD.
3048
out of
3820
Miroku Corporation
3049
out of
3820
KOKUYO CO.,LTD.
3050
out of
3820
NEPON Inc.
3051
out of
3820
NIHON I

3237
out of
3820
The Hachijuni Bank,Ltd.
3238
out of
3820
The Yamanashi Chuo Bank,Ltd.
3239
out of
3820
The Ogaki Kyoritsu Bank,Ltd.
3240
out of
3820
The Fukui Bank,Ltd.
3241
out of
3820
THE SHIMIZU BANK,LTD.
3242
out of
3820
The Bank of Toyama,Ltd.
3243
out of
3820
THE SHIGA BANK,LTD.
3244
out of
3820
The Nanto Bank,Ltd.
3245
out of
3820
The Hyakugo Bank,Ltd.
3246
out of
3820
The Bank of Kyoto,Ltd.
3247
out of
3820
The Kiyo Bank,Ltd.
3248
out of
3820
Hokuhoku Financial Group, Inc.
3249
out of
3820
The San-in Godo Bank,Ltd.
3250
out of
3820
The Chugoku Bank,Limited
3251
out of
3820
THE TOTTORI BANK,LTD.
3252
out of
3820
The Iyo Bank,Ltd.
3253
out of
3820
The Hyakujushi Bank,Ltd.
3254
out of
3820
The Shikoku Bank Ltd.
3255
out of
3820
The Awa Bank,Ltd.
3256
out of
3820
THE OITA BANK,LTD.
3257
out of
3820
The Miyazaki Bank,Ltd.
3258
out of
3820
THE BANK OF SAGA LTD.
3259
out of
3820
Bank of The Ryukyus,Limited
3260
out of
3820
Seven Bank,Ltd.
3261
out of
3820
Mizuho Financial Group,Inc.


3436
out of
3820
NISSIN CORPORATION
3437
out of
3820
MARUWN CORPORATION
3438
out of
3820
Maruzen Showa Unyu Co.,Ltd.
3439
out of
3820
SENKO Group Holdings Co.,Ltd.
3440
out of
3820
TONAMI HOLDINGS CO.,LTD.
3441
out of
3820
NIKKON Holdings Co.,Ltd.
3442
out of
3820
Kyogoku unyu shoji Co.,Ltd.
3443
out of
3820
Japan Oil Transportation Co.,Ltd.
3444
out of
3820
FUKUYAMA TRANSPORTING CO.,LTD.
3445
out of
3820
SEINO HOLDINGS CO.,LTD.
3446
out of
3820
S LINE CO.,LTD.
3447
out of
3820
Kanagawa Chuo Kotsu Co.,Ltd.
3448
out of
3820
Daiwa Motor Transportation Co.,Ltd.
3449
out of
3820
Shinki Bus Co.,Ltd.
3450
out of
3820
Hitachi Transport System,Ltd.
3451
out of
3820
TAKASE CORPORATION
3452
out of
3820
MARUWA UNYU KIKAN CO.,LTD.
3453
out of
3820
Chilled & Frozen Logistics Holdings Co.,Ltd.
3454
out of
3820
Nippon Yusen Kabushiki Kaisha
3455
out of
3820
Mitsui O.S.K.Lines,Ltd.
3456
out of
3820
Kawasaki Kisen Kaisha,Ltd.
3457
out of
3820
NS United Kaiun Kaisha,Ltd.
3458
out of
3820
MEIJI SHIPPING 

3633
out of
3820
TOHO CO.,LTD
3634
out of
3820
H.I.S.Co.,Ltd.
3635
out of
3820
TOEI COMPANY,LTD.
3636
out of
3820
WILSON LEARNING WORLDWIDE INC.
3637
out of
3820
LUCKLAND CO.,LTD.
3638
out of
3820
NTT DATA CORPORATION
3639
out of
3820
KYORITSU MAINTENANCE CO.,LTD.
3640
out of
3820
ICHINEN HOLDINGS CO.,LTD.
3641
out of
3820
CTI Engineering Co.,Ltd.
3642
out of
3820
SPACE CO.,LTD.
3643
out of
3820
CERESPO CO.,LTD.
3644
out of
3820
AIN HOLDINGS INC.
3645
out of
3820
SAN HOLDINGS,INC.
3646
out of
3820
PCA CORPORATION
3647
out of
3820
TOKYU RECREATION CO.,LTD.
3648
out of
3820
Subaru Enterprise Co.,Ltd.
3649
out of
3820
TOKYO THEATRES COMPANY,INCORPORATED
3650
out of
3820
Musashino Kogyo Co.,Ltd.
3651
out of
3820
Kin-Ei Corp.
3652
out of
3820
O S CO.,LTD.
3653
out of
3820
SANKYO FRONTIER CO.,LTD.
3654
out of
3820
SAISON INFORMATION SYSTEMS CO.,LTD.
3655
out of
3820
SACOS CORPORATION
3656
out of
3820
TANABE CONSULTING CO.,LTD.
3657
out of
3820
KYOWA ENGINEERING CONSULTANTS CO.,LTD.
3658
out 

In [ ]:
df = pd.read_excel (r'C:\Users\user\Desktop\Japan\Japan Company List - Completed.xlsx')
path = r'C:\Users\user\Desktop\자료\chromedriver'
cover = open('Japan2022 condensed 1.csv','w', newline='', encoding='UTF8')
writing = csv.writer(cover)
writing.writerow(['영문기업명','영문한줄소개내용','영문기업소개내용','기업홈페이지URL','영문기업주소','영문기업상세주소','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','영문산업군명','영문주요제품명내용'])
# Basic Setting
failList = []
driver = webdriver.Chrome(path)
driver.get('https://finance.yahoo.com/quote/1301.T/holders?p=1301.T')
time.sleep(3)
# Range must be changed
for i in range (2614,3000):
    try:
        # Search Setting
        name = df['현지언어기업명'].iloc[i]
        info = []
        # Status Check
        print(i)
        print(name)

        # Enter name of company in searchbox, and wait for 2 seconds.
        try:
            driver.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div[1]/div/div[2]/div/form/input[1]").send_keys(df['현지언어기업명'].iloc[i])
        except:
            driver.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div/div/div[1]/div/div[2]/div/form/input[1]").send_keys(df['현지언어기업명'].iloc[i])
        time.sleep(2)
        # Get to the company information page
        try:
            driver.find_element(By.XPATH, value='//*[@id="result-quotes-0"]/div[1]').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[1]/div/div/div/div/div/div/div/div/div/div/div/div/div[2]/div/div[2]/form/div/div[1]/div/ul[1]/li[1]').click()
        time.sleep(2)
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/div/div/section/div/ul/li[6]/a').click()
        time.sleep(2)
        
        #Company Identification
        info.append(name)

        #Descripton, Contact, Address, Extra Information, Management Information
        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan, Asia"
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan, Asia"
            except:
                try:
                    address1 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[1]').text + ", Japan"
                    address2 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[1]').text + ", Japan, Asia"
                except:
                    address1 = "Japan"
                    address2 = "Japan, Asia"
        try:
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[2]/p').text)
        except:
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group). Located at " + address2 + ", they are providing services related to  " + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[4]').text + ". You can find more information on company's website, " + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[2]').text)
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[2]').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul').text) 
            except:
                info.append("")
        info.append(address1)
        info.append(address2)

        # Financial Information
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul/li[7]/a').click()
        time.sleep(2)
        #회계연도

        try:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[2]/a/div').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[2]/a/div/span').click()
        time.sleep(2)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button').click()
        except:
            time.sleep(1)
            try: 
                driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/svg').click()
            except:
                try:
                    time.sleep(1)
                    driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/svg/path').click()
                except:
                    driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/div').click()
        time.sleep(2)

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[1]/div/div[2]').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[1]/div/div[2]/span').text) 
            except:
                info.append("")

        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[2]/div[1]/div[1]/div[2]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[2]/div[1]/div[1]/div[2]/span').text  + "K Yen")
            except:
                info.append("")
        #비유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[2]/div[2]/div[1]/div[2]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[2]/div[2]/div[1]/div[2]/span').text  + "K Yen")
            except:
                info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[1]/div[2]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[1]/div[2]/span').text  + "K Yen")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[2]/div[1]/div[1]/div[2]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[2]/div[1]/div[1]/div[2]/span').text  + "K Yen")
            except:
                info.append("")
        #비유동부채금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[2]/div[2]/div[1]/div[2]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[2]/div[2]/div[1]/div[2]/span').text + "K Yen")
            except:
                info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[1]/div[2]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[1]/div[2]/span').text + "K Yen")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[2]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[2]/span').text + "K Yen")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[3]/div[1]/div[2]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[3]/div[1]/div[2]/span').text  + "K Yen")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[1]/a').click()
        time.sleep(4)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[2]/div[1]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[5]/div[1]/div[3]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[5]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[6]/div[2]/div[1]/div[1]/div[3]/span').text + "K Yen")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[6]/div[2]/div[1]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #금융비용금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[6]/div[2]/div[2]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[6]/div[2]/div[2]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #기타영업외수익금액
        info.append("")
        #기타영업외비용금액
        info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[7]/div[1]/div[3]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[23]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[8]/div[1]/div[3]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[8]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[9]/div[2]/div[1]/div[2]/div[1]/div[2]/div/div[1]/div[3]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[27]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[3]/a').click()
        time.sleep(4)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[3]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[3]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[2]/div[3]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[2]/div[2]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")

        try:
            # Industry
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul/li[6]/a').click()
            time.sleep(2)
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[2]').text)
            except:
                info.append("General Business")
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[4]').text)
            except:
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[2]').text)
                except:
                    info.append("Product and Services")
        except:
            info.append("General Business")
            info.append("Product and Services")
        # Status Management
        print(info)
        writing.writerow(info)
    except:
        print("fail")

print("DONE")

C:\Users\user\AppData\Local\Temp\ipykernel_13328\3098701395.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


2614
MUSASHI SEIMITSU INDUSTRY CO.,LTD.


C:\Users\user\AppData\Local\Temp\ipykernel_13328\3098701395.py:23: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div[1]/div/div[2]/div/form/input[1]").send_keys(df['현지언어기업명'].iloc[i])
C:\Users\user\AppData\Local\Temp\ipykernel_13328\3098701395.py:25: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div/div/div[1]/div/div[2]/div/form/input[1]").send_keys(df['현지언어기업명'].iloc[i])


['MUSASHI SEIMITSU INDUSTRY CO.,LTD.', 'MUSASHI SEIMITSU INDUSTRY CO.,LTD.(English: MUSASHI SEIMITSU INDUSTRY CO.,LTD.) is a public company that is listed on JPX(Japan Exchange Group).', 'Musashi Seimitsu Industry Co., Ltd. manufactures and sells transportation equipment in Japan and internationally. The company provides parts for automobiles, such as power train, and linkage and suspension products. It also manufactures and supplies transmission gears, camshafts, and other drive parts for motorcycles, general-purpose engines, and all-terrain vehicles. Musashi Seimitsu Industry Co., Ltd. has a strategic collaboration with Aquarius Engines (A.M) Ltd. for research and development, assembly, distribution, marketing, and sales of the Aquarius engines and generators. The company was formerly known as Musashi Sangyo and changed its name to Musashi Seimitsu Industry Co., Ltd. in September 1963. Musashi Seimitsu Industry Co., Ltd. was founded in 1938 and is headquartered in Toyohashi, Japan.',

['YUTAKA GIKEN CO.,LTD.', 'YUTAKA GIKEN CO.,LTD.(English: YUTAKA GIKEN CO.,LTD.) is a public company that is listed on JPX(Japan Exchange Group).', 'Yutaka Giken Co.,Ltd. manufactures and sells automobile parts in Japan and internationally. Its products include torque converters; exhaust system products, such as exhaust manifolds, catalytic converters, silencers, and heat collectors; brake discs for motorcycles; and motor parts, including stator cores and rotors. The company was founded in 1954 and is headquartered in Hamamatsu, Japan. Yutaka Giken Co.,Ltd. is a subsidiary of Honda Motor Co., Ltd.', 'https://www.yutakagiken.co.jp', 'Japan', 'Japan, Asia', '3/30/2022', '150,588,000K Yen', '57,746,000K Yen', '208,334,000K Yen', '108,907,000K Yen', '4,453,000K Yen', '84,460,000K Yen', '', '94,974,000K Yen', '213,395,000K Yen', '190,725,000K Yen', '15,278,000K Yen', '8,854,000K Yen', '2,617,000K Yen', '105,000K Yen', '', '', '', '', '', '12,142,000K Yen', '-3,947,000K Yen', '-2,793,000K Ye

...

['OZU CORPORATION', 'OZU CORPORATION(English: OZU CORPORATION) is a public company that is listed on JPX(Japan Exchange Group).', 'Ozu Corporation imports, processes, sells, and exports nonwoven fabrics for electronics, medical use, cosmetics, and industrial materials in Japan and internationally. It also offers paper and paper products. The company was founded in 1653 and is headquartered in Tokyo, Japan.', 'https://www.ozu.co.jp', 'Japan', 'Japan, Asia', '5/30/2021', '11,904,370K Yen', '10,409,940K Yen', '22,314,313K Yen', '5,055,608K Yen', '1,266,453K Yen', '16,183,121K Yen', '', '15,992,252K Yen', '33,922,714K Yen', '28,265,158K Yen', '1,512,906K Yen', '868,686K Yen', '2,745K Yen', '13,618K Yen', '', '', '816,826K Yen', '', '724,171K Yen', '1,318,173K Yen', '3,110,792K Yen', '-375,955K Yen', '-6,571K Yen', '6,616,151K Yen', 'Basic Materials', 'Paper & Paper Products']
2764
NISSIN SHOJI CO.,LTD.
['NISSIN SHOJI CO.,LTD.', 'NISSIN SHOJI CO.,LTD.(English: NISSIN SHOJI CO.,LTD.) is a pu

['HOUSE OF ROSE Co.,Ltd.', 'HOUSE OF ROSE Co.,Ltd.(English: HOUSE OF ROSE Co.,Ltd.) is a public company that is listed on JPX(Japan Exchange Group).', "HOUSE OF ROSE Co.,Ltd. operates cosmetic stores in Japan. The company's stores offer skin care, makeup, body and hair care, bath, fragrant body care cosmetics, delicately fragranced sundry goods, and other cosmetic products. It also operates relaxation salons and fitness salons for women under the Curves brand name. As of March 31, 2022, the company operated 248 stores, including 204 stores under the HOUSE OF ROSE name; 21 stores under the Curves name; 17 relaxation salons; and 6 other stores. HOUSE OF ROSE Co.,Ltd. was founded in 1978 and is headquartered in Tokyo, Japan.", 'https://www.houseofrose.co.jp', 'Japan', 'Japan, Asia', '3/30/2022', '5,605,726K Yen', '3,332,493K Yen', '8,938,220K Yen', '2,196,125K Yen', '1,416,832K Yen', '5,425,263K Yen', '', '5,325,263K Yen', '11,283,938K Yen', '3,186,976K Yen', '1,140,213K Yen', '262,634K Y

['HIMARAYA Co.,Ltd.', 'HIMARAYA Co.,Ltd.(English: HIMARAYA Co.,Ltd.) is a public company that is listed on JPX(Japan Exchange Group).', 'HIMARAYA Co.,Ltd. engages in the retail of sporting goods in Japan. It offers apparels, shoes, and outdoor apparels; fitness/yoga and hiking/trekking products, as well as ski snowboards; and sporting goods of various genres, such as tennis, baseball, soccer, golf marine sports, and outdoor equipment. The company was formerly known as Himaraya, Ltd. and changed its name to HIMARAYA Co.,Ltd. in 1991. HIMARAYA Co.,Ltd. was founded in 1976 and is headquartered in Gifu, Japan.', 'https://www.himaraya.co.jp', 'Japan', 'Japan, Asia', '8/30/2021', '29,410,000K Yen', '10,633,000K Yen', '40,048,000K Yen', '15,559,000K Yen', '9,370,000K Yen', '23,349,000K Yen', '', '15,119,000K Yen', '62,133,000K Yen', '39,883,000K Yen', '', '2,024,000K Yen', '9,000K Yen', '33,000K Yen', '', '', '', '', '-65,000K Yen', '6,894,000K Yen', '-148,000K Yen', '-2,908,000K Yen', '9,915

['WATAMI CO.,LTD.', 'WATAMI CO.,LTD.(English: WATAMI CO.,LTD.) is a public company that is listed on JPX(Japan Exchange Group).', 'Watami Co., Ltd. engages in the food, home food, and agriculture services business in Japan and internationally. The company operates restaurants. It also engages in organic farming, livestock, and dairy businesses; and development, construction, operation, and management of wind power generation facilities. The company was incorporated in 1967 and is headquartered in Tokyo, Japan.', 'https://www.watami.co.jp', 'Japan', 'Japan, Asia', '3/30/2022', '37,660,000K Yen', '14,560,000K Yen', '52,223,000K Yen', '14,837,000K Yen', '22,754,000K Yen', '31,634,000K Yen', '', '14,632,000K Yen', '64,362,000K Yen', '29,723,000K Yen', '', '-3,578,000K Yen', '32,000K Yen', '278,000K Yen', '', '', '', '', '-4,215,000K Yen', '1,019,000K Yen', '-15,365,000K Yen', '10,103,000K Yen', '333,000K Yen', '10,971,000K Yen', 'Consumer Cyclical', 'Restaurants']
2784
ART VIVANT CO.,LTD.


['DAISUI CO.,LTD.', 'DAISUI CO.,LTD.(English: DAISUI CO.,LTD.) is a public company that is listed on JPX(Japan Exchange Group).', 'Daisui Co.,Ltd. engages in the wholesale of fresh and frozen fish products in Japan. It also provides contract refrigeration and warehousing services. The company exports its products worldwide. Daisui Co.,Ltd. was founded in 1939 and is headquartered in Osaka, Japan.', 'https://www.daisui.co.jp', 'Japan', 'Japan, Asia', '3/30/2022', '15,715,000K Yen', '5,352,000K Yen', '21,070,000K Yen', '9,747,000K Yen', '3,770,000K Yen', '8,753,000K Yen', '', '7,553,000K Yen', '88,788,000K Yen', '82,854,000K Yen', '', '-190,000K Yen', '6,000K Yen', '20,000K Yen', '', '', '', '', '104,000K Yen', '-2,000,000K Yen', '280,000K Yen', '428,000K Yen', '3,375,000K Yen', '2,083,000K Yen', 'Consumer Defensive', 'Food Distribution']
2792
AINAVO HOLDINGS Co.,Ltd.
fail
2793
Three F Co.,Ltd.
fail
2794
NISHIMATSUYA CHAIN Co.,Ltd.
fail
2795
ZENSHO HOLDINGS CO.,LTD.
fail
2796
WEDS CO.,LT

### 2021

In [ ]:
df = pd.read_excel (r'C:\Users\user\Desktop\Japan\Japan Company List - Completed.xlsx')
path = r'C:\Users\user\Desktop\자료\chromedriver'
cover = open('Japan2021.csv','w', newline='', encoding='UTF8')
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드','현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])
# Basic Setting
failList = []
driver = webdriver.Chrome(path)
driver.get('https://finance.yahoo.com/quote/1301.T/holders?p=1301.T')
time.sleep(3)
# Range must be changed
for i in range (0,3819):
    try:
        # Search Setting
        time.sleep(1)  
        name = df['현지언어기업명'].iloc[i]
        info = []
        # Status Check
        print(i)
        print("out of")
        print(3820)
        print(name)

        # Enter name of company in searchbox, and wait for 2 seconds.
        try:
            driver.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div[1]/div/div[2]/div/form/input[1]").send_keys(df['현지언어기업명'].iloc[i])
        except:
            driver.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div/div/div[1]/div/div[2]/div/form/input[1]").send_keys(df['현지언어기업명'].iloc[i])
        time.sleep(3)
        # Get to the company information page
        try:
            driver.find_element(By.XPATH, value='//*[@id="result-quotes-0"]/div[1]').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[1]/div/div/div/div/div/div/div/div/div/div/div/div/div[2]/div/div[2]/form/div/div[1]/div/ul[1]/li[1]').click()
        time.sleep(4)
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/div/div/section/div/ul/li[6]/a').click()
        time.sleep(4)

        #Country Information
        info.append("JPN") 
        info.append("Nihon") 
        info.append("Japan") 
        info.append("UTC+09:00") 
        info.append("아시아") 
        info.append("6100000000000 USD") 
        info.append("125502000") 
        info.append("동북아시아") 

        #Company Identification
        info.append("JPN202206" + name)
        info.append(name)
        info.append(name)

        #Descripton, Contact, Address, Extra Information, Management Information
        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan, Asia"
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan, Asia"
            except:
                try:
                    address1 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[1]').text + ", Japan"
                    address2 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[1]').text + ", Japan, Asia"
                except:
                    address1 = "Japan"
                    address2 = "Japan, Asia"
        try:
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[2]/p').text) 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[2]/p').text) 
        except:
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group). Located at " + address2 + ", they are providing services related to  " + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[4]').text + ". You can find more information on company's website, " + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[2]').text)
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group). Located at " + address2 + ", they are providing services related to  " + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[4]').text + ". You can find more information on company's website, " + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[2]').text)
        info.append("")
        info.append("")
        info.append("")

        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[1]').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[2]').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul').text) 
            except:
                info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(address1)
        info.append(address1)
        info.append(address2)
        info.append(address2)
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(df['기업상태'].iloc[i]) 
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[2]/span').text)
        except:
            info.append("Contact")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[2]/span').text)
        except:
            info.append("Contact")
        try:
            info.append("Executive Board")
        except:
            info.append("Contact")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[1]').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[1]').text) 
        except:
            info.append("")

        # Financial Information
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul/li[7]/a').click()
        time.sleep(4)
        #회계연도

        try:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[2]/a/div').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[2]/a/div/span').click()
        time.sleep(2)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button').click()
        except:
            time.sleep(2)
            try: 
                driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/svg').click()
            except:
                try:
                    time.sleep(2)
                    driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/svg/path').click()
                except:
                    driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/div').click()
        time.sleep(2)

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[1]/div/div[3]/span').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[1]/div/div[3]/span').text) 
            except:
                info.append("")

        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[2]/div[1]/div[1]/div[3]/span').text  + "K Yen")
            except:
                info.append("")
        #비유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[2]/div[2]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[2]/div[2]/div[1]/div[3]/span').text  + "K Yen")
            except:
                info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[1]/div[3]/span').text  + "K Yen")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[2]/div[1]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[2]/div[1]/div[1]/div[3]/span').text  + "K Yen")
            except:
                info.append("")
        #비유동부채금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[2]/div[2]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[2]/div[2]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[3]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[3]/div[1]/div[3]/span').text  + "K Yen")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[1]/a').click()
        time.sleep(4)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[2]/div[1]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[5]/div[1]/div[4]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[5]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[6]/div[2]/div[1]/div[1]/div[4]/span').text + "K Yen")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[6]/div[2]/div[1]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #금융비용금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[6]/div[2]/div[2]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[6]/div[2]/div[2]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #기타영업외수익금액
        info.append("")
        #기타영업외비용금액
        info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[7]/div[1]/div[4]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[23]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[8]/div[1]/div[4]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[8]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[9]/div[2]/div[1]/div[2]/div[1]/div[2]/div/div[1]/div[4]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[27]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[3]/a').click()
        time.sleep(4)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[3]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[3]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[2]/div[3]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[2]/div[2]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #재무정보중 계산예정
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")



        # Industry
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul/li[6]/a').click()
        time.sleep(4)
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[2]').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[2]').text)
        except:
            info.append("General Business")
            info.append("General Business")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[4]').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[4]').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[2]').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[2]').text)
            except:
                info.append("Product and Services")
                info.append("Product and Services")


        # Language Information
        info.append("JPN")
        info.append("Nihongo")
        info.append("Japanese")

        # Stock Market Information
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul/li[1]/a').click()
        time.sleep(4)
        info.append("JPX")
        info.append("Japan Exchange Group")
        info.append("Japan Exchange Group")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[6]/div/div[2]/div[1]/div[1]/h1').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[6]/div/div[2]/div[1]/div[1]').text)
            except:
                info.append("")
        #상장일자
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/div/div[2]/div[1]/table/tbody/tr[5]/td[2]').text + " JPY")
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/div/div[2]/div[1]/table/tbody/tr[6]/td[2]').text + " JPY")
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        #거래량
        info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/div/div[2]/div[1]/table/tbody/tr[7]/td[2]/fin-streamer').text)
        #시가총액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/div/div[2]/div[2]/table/tbody/tr[1]/td[2]').text)
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Japan")
        info.append("Japan, Asia")
        info.append("")
        info.append("")

        # Event
        info.append("Not Available")

        # Currency Information
        info.append("JPY")
        info.append("Yen")

        # Management
        info.append("Chris")
        info.append("Yahoo Finance")
        info.append("2022-06-21")

        # Status Management
        print(info)
        writing.writerow(info)
    except:
        try:
            info = []
            name = df['현지언어기업명'].iloc[i]
            #국가코드
            info.append("JPN")
            info.append("Nihon")
            info.append("Japan")
            info.append("UTC+09:00")
            info.append("아시아")
            info.append("6100000000000 USD")
            info.append("125502000")
            info.append("동북아시아")
            info.append("JPN202206" + name)
            info.append(name)
            info.append(name)
            address1 = "Japan"
            address2 = "Japan, Asia"
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group). Located at " + address2 + ", they are operated as public company. You can find more information on company's website.")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group). Located at " + address2 + ", they are operated as public company. You can find more information on company's website.")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append(address1)
            info.append(address1)
            info.append(address2)
            info.append(address2)
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append(df['기업상태'].iloc[i]) 
            info.append("")
            info.append("")
            info.append("Contact")
            info.append("Contact")
            info.append("Contact")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("TTM")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("General Business")
            info.append("General Business")
            info.append("Product and Services")
            info.append("Product and Services")
            info.append("JPN")
            info.append("Nihongo")
            info.append("Japanese")
            info.append("JPX")
            info.append("Japan Exchange Group")
            info.append("Japan Exchange Group")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Japan")
            info.append("Japan, Asia")
            info.append("")
            info.append("")
            info.append("Not Available")
            info.append("JPY")
            info.append("Yen")
            info.append("Chris")
            info.append("Yahoo Finance")
            info.append("2022-06-21")

            writing.writerow(info)
        except:
            failList.append(df['현지언어기업명'].iloc[i])
            
# Check failed result 
# - There are many ways to deal with failList: retry, ignore, manually add, find why error has occured.
# - This will be up to user's choice.
print("failLength:")
print(len(failList))
print(failList)

In [ ]:
df = pd.read_excel (r'C:\Users\user\Desktop\Japan\Japan Company List - Completed.xlsx')
path = r'C:\Users\user\Desktop\자료\chromedriver'
cover = open('Japan2021 condensed.csv','w', newline='', encoding='UTF8')
writing = csv.writer(cover)
writing.writerow(['영문기업명','영문한줄소개내용','영문기업소개내용','기업홈페이지URL','영문기업주소','영문기업상세주소','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','영문산업군명','영문주요제품명내용'])
# Basic Setting
failList = []
driver = webdriver.Chrome(path)
driver.get('https://finance.yahoo.com/quote/1301.T/holders?p=1301.T')
time.sleep(3)
# Range must be changed
for i in range (0,3819):
    try:
        # Search Setting
        name = df['현지언어기업명'].iloc[i]
        info = []
        # Status Check
        print(i)
        print(name)

        # Enter name of company in searchbox, and wait for 2 seconds.
        try:
            driver.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div[1]/div/div[2]/div/form/input[1]").send_keys(df['현지언어기업명'].iloc[i])
        except:
            driver.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div/div/div[1]/div/div[2]/div/form/input[1]").send_keys(df['현지언어기업명'].iloc[i])
        time.sleep(2)
        # Get to the company information page
        try:
            driver.find_element(By.XPATH, value='//*[@id="result-quotes-0"]/div[1]').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[1]/div/div/div/div/div/div/div/div/div/div/div/div/div[2]/div/div[2]/form/div/div[1]/div/ul[1]/li[1]').click()
        time.sleep(2)
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/div/div/section/div/ul/li[6]/a').click()
        time.sleep(2)
        
        #Company Identification
        info.append(name)

        #Descripton, Contact, Address, Extra Information, Management Information
        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan, Asia"
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan, Asia"
            except:
                try:
                    address1 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[1]').text + ", Japan"
                    address2 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[1]').text + ", Japan, Asia"
                except:
                    address1 = "Japan"
                    address2 = "Japan, Asia"
        try:
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[2]/p').text)
        except:
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group). Located at " + address2 + ", they are providing services related to  " + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[4]').text + ". You can find more information on company's website, " + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[2]').text)
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[2]').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul').text) 
            except:
                info.append("")
        info.append(address1)
        info.append(address2)

        # Financial Information
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul/li[7]/a').click()
        time.sleep(2)
        #회계연도

        try:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[2]/a/div').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[2]/a/div/span').click()
        time.sleep(2)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button').click()
        except:
            time.sleep(1)
            try: 
                driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/svg').click()
            except:
                try:
                    time.sleep(1)
                    driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/svg/path').click()
                except:
                    driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/div').click()
        time.sleep(2)

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[1]/div/div[3]/span').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[1]/div/div[3]/span').text) 
            except:
                info.append("")

        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[2]/div[1]/div[1]/div[3]/span').text  + "K Yen")
            except:
                info.append("")
        #비유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[2]/div[2]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[2]/div[2]/div[1]/div[3]/span').text  + "K Yen")
            except:
                info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[1]/div[3]/span').text  + "K Yen")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[2]/div[1]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[2]/div[1]/div[1]/div[3]/span').text  + "K Yen")
            except:
                info.append("")
        #비유동부채금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[2]/div[2]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[2]/div[2]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[3]/span').text + "K Yen")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[3]/div[1]/div[3]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[3]/div[1]/div[3]/span').text  + "K Yen")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[1]/a').click()
        time.sleep(2)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[2]/div[1]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[5]/div[1]/div[4]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[5]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[6]/div[2]/div[1]/div[1]/div[4]/span').text + "K Yen")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[6]/div[2]/div[1]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #금융비용금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[6]/div[2]/div[2]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[6]/div[2]/div[2]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[7]/div[1]/div[4]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[23]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[8]/div[1]/div[4]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[8]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[9]/div[2]/div[1]/div[2]/div[1]/div[2]/div/div[1]/div[4]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[27]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[3]/a').click()
        time.sleep(2)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[3]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[3]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[2]/div[3]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[2]/div[2]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")

        try:
            # Industry
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul/li[6]/a').click()
            time.sleep(2)
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[2]').text)
            except:
                info.append("General Business")
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[4]').text)
            except:
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[2]').text)
                except:
                    info.append("Product and Services")
        except:
            info.append("General Business")
            info.append("Product and Services")
        # Status Management
        print(info)
        writing.writerow(info)
    except:
        print("fail")

print("DONE")

### 2020

In [ ]:
df = pd.read_excel (r'C:\Users\user\Desktop\Japan\Japan Company List - Completed.xlsx')
path = r'C:\Users\user\Desktop\자료\chromedriver'
cover = open('Japan2020.csv','w', newline='', encoding='UTF8')
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드','현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])
# Basic Setting
failList = []
driver = webdriver.Chrome(path)
driver.get('https://finance.yahoo.com/quote/1301.T/holders?p=1301.T')
time.sleep(3)
# Range must be changed
for i in range (0,3819):
    try:
        # Search Setting
        time.sleep(1)  
        name = df['현지언어기업명'].iloc[i]
        info = []
        # Status Check
        print(i)
        print("out of")
        print(3820)
        print(name)

        # Enter name of company in searchbox, and wait for 2 seconds.
        try:
            driver.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div[1]/div/div[2]/div/form/input[1]").send_keys(df['현지언어기업명'].iloc[i])
        except:
            driver.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div/div/div[1]/div/div[2]/div/form/input[1]").send_keys(df['현지언어기업명'].iloc[i])
        time.sleep(3)
        # Get to the company information page
        try:
            driver.find_element(By.XPATH, value='//*[@id="result-quotes-0"]/div[1]').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[1]/div/div/div/div/div/div/div/div/div/div/div/div/div[2]/div/div[2]/form/div/div[1]/div/ul[1]/li[1]').click()
        time.sleep(4)
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/div/div/section/div/ul/li[6]/a').click()
        time.sleep(4)

        #Country Information
        info.append("JPN") 
        info.append("Nihon") 
        info.append("Japan") 
        info.append("UTC+09:00") 
        info.append("아시아") 
        info.append("6100000000000 USD") 
        info.append("125502000") 
        info.append("동북아시아") 
        
        #Company Identification
        info.append("JPN202206" + name)
        info.append(name)
        info.append(name)

        #Descripton, Contact, Address, Extra Information, Management Information
        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan, Asia"
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan, Asia"
            except:
                try:
                    address1 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[1]').text + ", Japan"
                    address2 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[1]').text + ", Japan, Asia"
                except:
                    address1 = "Japan"
                    address2 = "Japan, Asia"
        try:
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[2]/p').text) 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[2]/p').text) 
        except:
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group). Located at " + address2 + ", they are providing services related to  " + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[4]').text + ". You can find more information on company's website, " + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[2]').text)
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group). Located at " + address2 + ", they are providing services related to  " + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[4]').text + ". You can find more information on company's website, " + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[2]').text)
        info.append("")
        info.append("")
        info.append("")

        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[1]').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[2]').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul').text) 
            except:
                info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(address1)
        info.append(address1)
        info.append(address2)
        info.append(address2)
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(df['기업상태'].iloc[i]) 
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[2]/span').text)
        except:
            info.append("Contact")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[2]/span').text)
        except:
            info.append("Contact")
        try:
            info.append("Executive Board")
        except:
            info.append("Contact")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[1]').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[1]').text) 
        except:
            info.append("")

        # Financial Information
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul/li[7]/a').click()
        time.sleep(4)
        #회계연도

        try:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[2]/a/div').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[2]/a/div/span').click()
        time.sleep(2)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button').click()
        except:
            time.sleep(2)
            try: 
                driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/svg').click()
            except:
                try:
                    time.sleep(2)
                    driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/svg/path').click()
                except:
                    driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/div').click()
        time.sleep(2)

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[1]/div/div[4]/span').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[1]/div/div[4]/span').text) 
            except:
                info.append("")

        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[2]/div[1]/div[1]/div[4]/span').text  + "K Yen")
            except:
                info.append("")
        #비유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[2]/div[2]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[2]/div[2]/div[1]/div[4]/span').text  + "K Yen")
            except:
                info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[1]/div[4]/span').text  + "K Yen")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[2]/div[1]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[2]/div[1]/div[1]/div[4]/span').text  + "K Yen")
            except:
                info.append("")
        #비유동부채금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[2]/div[2]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[2]/div[2]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[3]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[3]/div[1]/div[4]/span').text  + "K Yen")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[1]/a').click()
        time.sleep(4)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[2]/div[1]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[5]/div[1]/div[5]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[5]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[6]/div[2]/div[1]/div[1]/div[5]/span').text + "K Yen")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[6]/div[2]/div[1]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #금융비용금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[6]/div[2]/div[2]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[6]/div[2]/div[2]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #기타영업외수익금액
        info.append("")
        #기타영업외비용금액
        info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[7]/div[1]/div[5]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[23]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[8]/div[1]/div[5]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[8]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[9]/div[2]/div[1]/div[2]/div[1]/div[2]/div/div[1]/div[5]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[27]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[3]/a').click()
        time.sleep(4)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[3]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[3]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[2]/div[3]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[2]/div[2]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #재무정보중 계산예정
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")


        # Industry
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul/li[6]/a').click()
        time.sleep(4)
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[2]').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[2]').text)
        except:
            info.append("General Business")
            info.append("General Business")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[4]').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[4]').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[2]').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[2]').text)
            except:
                info.append("Product and Services")
                info.append("Product and Services")

        # Status Management
        print(info)
        writing.writerow(info)
    except:
        try:
            info = []
            name = df['현지언어기업명'].iloc[i]
            #국가코드
            info.append("JPN")
            info.append("Nihon")
            info.append("Japan")
            info.append("UTC+09:00")
            info.append("아시아")
            info.append("6100000000000 USD")
            info.append("125502000")
            info.append("동북아시아")
            info.append("JPN202206" + name)
            info.append(name)
            info.append(name)
            address1 = "Japan"
            address2 = "Japan, Asia"
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group). Located at " + address2 + ", they are operated as public company. You can find more information on company's website.")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group). Located at " + address2 + ", they are operated as public company. You can find more information on company's website.")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append(address1)
            info.append(address1)
            info.append(address2)
            info.append(address2)
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append(df['기업상태'].iloc[i]) 
            info.append("")
            info.append("")
            info.append("Contact")
            info.append("Contact")
            info.append("Contact")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("TTM")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("General Business")
            info.append("General Business")
            info.append("Product and Services")
            info.append("Product and Services")
            info.append("JPN")
            info.append("Nihongo")
            info.append("Japanese")
            info.append("JPX")
            info.append("Japan Exchange Group")
            info.append("Japan Exchange Group")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Japan")
            info.append("Japan, Asia")
            info.append("")
            info.append("")
            info.append("Not Available")
            info.append("JPY")
            info.append("Yen")
            info.append("Chris")
            info.append("Yahoo Finance")
            info.append("2022-06-21")

            writing.writerow(info)
        except:
            failList.append(df['현지언어기업명'].iloc[i])
            
# Check failed result 
# - There are many ways to deal with failList: retry, ignore, manually add, find why error has occured.
# - This will be up to user's choice.
print("failLength:")
print(len(failList))
print(failList)

In [ ]:
df = pd.read_excel (r'C:\Users\user\Desktop\Japan\Japan Company List - Completed.xlsx')
path = r'C:\Users\user\Desktop\자료\chromedriver'
cover = open('Japan2020 condensed.csv','w', newline='', encoding='UTF8')
writing = csv.writer(cover)
writing.writerow(['영문기업명','영문한줄소개내용','영문기업소개내용','기업홈페이지URL','영문기업주소','영문기업상세주소','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','영문산업군명','영문주요제품명내용'])
# Basic Setting
failList = []
driver = webdriver.Chrome(path)
driver.get('https://finance.yahoo.com/quote/1301.T/holders?p=1301.T')
time.sleep(3)
# Range must be changed
for i in range (0,3819):
    try:
        # Search Setting
        name = df['현지언어기업명'].iloc[i]
        info = []
        # Status Check
        print(i)
        print(name)
                 
        # Enter name of company in searchbox, and wait for 2 seconds.
        try:
            driver.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div[1]/div/div[2]/div/form/input[1]").send_keys(df['현지언어기업명'].iloc[i])
        except:
            driver.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div/div/div[1]/div/div[2]/div/form/input[1]").send_keys(df['현지언어기업명'].iloc[i])
        time.sleep(2)
        # Get to the company information page
        try:
            driver.find_element(By.XPATH, value='//*[@id="result-quotes-0"]/div[1]').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[1]/div/div/div/div/div/div/div/div/div/div/div/div/div[2]/div/div[2]/form/div/div[1]/div/ul[1]/li[1]').click()
        time.sleep(2)
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/div/div/section/div/ul/li[6]/a').click()
        time.sleep(2)
        
        #Company Identification
        info.append(name)

        #Descripton, Contact, Address, Extra Information, Management Information
        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan, Asia"
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan, Asia"
            except:
                try:
                    address1 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[1]').text + ", Japan"
                    address2 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[1]').text + ", Japan, Asia"
                except:
                    address1 = "Japan"
                    address2 = "Japan, Asia"
        try:
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[2]/p').text)
        except:
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group). Located at " + address2 + ", they are providing services related to  " + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[4]').text + ". You can find more information on company's website, " + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[2]').text)
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[2]').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul').text) 
            except:
                info.append("")
        info.append(address1)
        info.append(address2)

        # Financial Information
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul/li[7]/a').click()
        time.sleep(2)
        #회계연도
        try:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[2]/a/div').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[2]/a/div/span').click()
        time.sleep(2)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button').click()
        except:
            try:
                time.sleep(1)
                driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/svg').click()
            except:
                try:
                    driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/svg/path').click()
                except:
                    driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/div').click()
        time.sleep(2)

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[1]/div/div[4]/span').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[1]/div/div[4]/span').text) 
            except:
                info.append("")

        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[2]/div[1]/div[1]/div[4]/span').text  + "K Yen")
            except:
                info.append("")
        #비유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[2]/div[2]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[2]/div[2]/div[1]/div[4]/span').text  + "K Yen")
            except:
                info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[1]/div[4]/span').text  + "K Yen")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[2]/div[1]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[2]/div[1]/div[1]/div[4]/span').text  + "K Yen")
            except:
                info.append("")
        #비유동부채금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[2]/div[2]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[2]/div[2]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[4]/span').text + "K Yen")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[3]/div[1]/div[4]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[3]/div[1]/div[4]/span').text  + "K Yen")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[1]/a').click()
        time.sleep(4)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[2]/div[1]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[5]/div[1]/div[5]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[5]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[6]/div[2]/div[1]/div[1]/div[5]/span').text + "K Yen")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[6]/div[2]/div[1]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #금융비용금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[6]/div[2]/div[2]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[6]/div[2]/div[2]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[7]/div[1]/div[5]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[23]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[8]/div[1]/div[5]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[8]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[9]/div[2]/div[1]/div[2]/div[1]/div[2]/div/div[1]/div[5]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[27]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[3]/a').click()
        time.sleep(4)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[3]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[3]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[2]/div[3]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[2]/div[2]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")

        try:
            # Industry
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul/li[6]/a').click()
            time.sleep(2)
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[2]').text)
            except:
                info.append("General Business")
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[4]').text)
            except:
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[2]').text)
                except:
                    info.append("Product and Services")
        except:
            info.append("General Business")
            info.append("Product and Services")
        # Status Management
        print(info)
        writing.writerow(info)
    except:
        print("fail")

print("DONE")

### 2019

In [ ]:
df = pd.read_excel (r'C:\Users\user\Desktop\Japan\Japan Company List - Completed.xlsx')
path = r'C:\Users\user\Desktop\자료\chromedriver'
cover = open('Japan2019.csv','w', newline='', encoding='UTF8')
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드','현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])
# Basic Setting
failList = []
driver = webdriver.Chrome(path)
driver.get('https://finance.yahoo.com/quote/1301.T/holders?p=1301.T')
time.sleep(3)
# Range must be changed
for i in range (0,3819):
    try:
        # Search Setting
        time.sleep(1)  
        name = df['현지언어기업명'].iloc[i]
        info = []
        # Status Check
        print(i)
        print("out of")
        print(3820)
        print(name)

        # Enter name of company in searchbox, and wait for 2 seconds.
        try:
            driver.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div[1]/div/div[2]/div/form/input[1]").send_keys(df['현지언어기업명'].iloc[i])
        except:
            driver.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div/div/div[1]/div/div[2]/div/form/input[1]").send_keys(df['현지언어기업명'].iloc[i])
        time.sleep(3)
        # Get to the company information page
        try:
            driver.find_element(By.XPATH, value='//*[@id="result-quotes-0"]/div[1]').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[1]/div/div/div/div/div/div/div/div/div/div/div/div/div[2]/div/div[2]/form/div/div[1]/div/ul[1]/li[1]').click()
        time.sleep(4)
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/div/div/section/div/ul/li[6]/a').click()
        time.sleep(4)

        #Country Information
        info.append("JPN") 
        info.append("Nihon") 
        info.append("Japan") 
        info.append("UTC+09:00") 
        info.append("아시아") 
        info.append("6100000000000 USD") 
        info.append("125502000") 
        info.append("동북아시아") 

        #Company Identification
        info.append("JPN202206" + name)
        info.append(name)
        info.append(name)

        #Descripton, Contact, Address, Extra Information, Management Information
        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan, Asia"
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan, Asia"
            except:
                try:
                    address1 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[1]').text + ", Japan"
                    address2 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[1]').text + ", Japan, Asia"
                except:
                    address1 = "Japan"
                    address2 = "Japan, Asia"
        try:
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[2]/p').text) 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[2]/p').text) 
        except:
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group). Located at " + address2 + ", they are providing services related to  " + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[4]').text + ". You can find more information on company's website, " + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[2]').text)
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group). Located at " + address2 + ", they are providing services related to  " + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[4]').text + ". You can find more information on company's website, " + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[2]').text)
        info.append("")
        info.append("")
        info.append("")

        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[1]').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[2]').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul').text) 
            except:
                info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(address1)
        info.append(address1)
        info.append(address2)
        info.append(address2)
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(df['기업상태'].iloc[i]) 
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[2]/span').text)
        except:
            info.append("Contact")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[2]/span').text)
        except:
            info.append("Contact")
        try:
            info.append("Executive Board")
        except:
            info.append("Contact")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[1]').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[1]').text) 
        except:
            info.append("")

        # Financial Information
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul/li[7]/a').click()
        time.sleep(4)
        #회계연도

        try:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[2]/a/div').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[2]/a/div/span').click()
        time.sleep(2)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button').click()
        except:
            time.sleep(2)
            try: 
                driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/svg').click()
            except:
                try:
                    time.sleep(2)
                    driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/svg/path').click()
                except:
                    driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/div').click()
        time.sleep(2)

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[1]/div/div[5]/span').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[1]/div/div[5]').text) 
            except:
                info.append("")

        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/div[1]/div[5]/span').text  + "K Yen")
            except:
                info.append("")
        #비유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[2]/div[2]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[2]/div[2]/div[1]/div[5]/span').text  + "K Yen")
            except:
                info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[1]/div[5]/span').text  + "K Yen")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[2]/div[1]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[2]/div[1]/div[1]/div[5]/span').text  + "K Yen")
            except:
                info.append("")
        #비유동부채금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[2]/div[2]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[2]/div[2]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[3]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[3]/div[1]/div[5]/span').text  + "K Yen")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[1]/a').click()
        time.sleep(4)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[1]/div[6]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[1]/div[6]').text + "K Yen")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[1]/div[6]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[1]/div[6]').text + "K Yen")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[6]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[6]').text + "K Yen")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[5]/div[1]/div[6]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[5]/div[1]/div[6]/span').text + "K Yen")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[6]/div[2]/div[1]/div[1]/div[6]/span').text + "K Yen")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[6]/div[2]/div[1]/div[1]/div[6]').text + "K Yen")
            except:
                info.append("")
        #금융비용금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[6]/div[2]/div[2]/div[1]/div[6]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[6]/div[2]/div[2]/div[1]/div[6]').text + "K Yen")
            except:
                info.append("")
        #기타영업외수익금액
        info.append("")
        #기타영업외비용금액
        info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[7]/div[1]/div[6]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[7]/div[1]/div[6]/span').text + "K Yen")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[8]/div[1]/div[6]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[8]/div[1]/div[6]/span').text + "K Yen")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[9]/div[2]/div[1]/div[2]/div[1]/div[2]/div/div[1]/div[6]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[9]/div[2]/div[1]/div[2]/div[1]/div[2]/div/div[1]/div[6]/span').text + "K Yen")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[3]/a').click()
        time.sleep(4)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[1]/div[6]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[1]/div[6]').text + "K Yen")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[1]/div[6]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[1]/div[6]').text + "K Yen")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[3]/div[1]/div[6]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[3]/div[1]/div[6]').text + "K Yen")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[2]/div[3]/div[1]/div[6]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[2]/div[3]/div[1]/div[6]').text + "K Yen")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[6]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[6]').text + "K Yen")
            except:
                info.append("")
        #재무정보중 계산예정
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")


        # Industry
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul/li[6]/a').click()
        time.sleep(4)
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[2]').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[2]').text)
        except:
            info.append("General Business")
            info.append("General Business")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[4]').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[4]').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[2]').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[2]').text)
            except:
                info.append("Product and Services")
                info.append("Product and Services")


        # Language Information
        info.append("JPN")
        info.append("Nihongo")
        info.append("Japanese")

        # Stock Market Information
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul/li[1]/a').click()
        time.sleep(4)
        info.append("JPX")
        info.append("Japan Exchange Group")
        info.append("Japan Exchange Group")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[6]/div/div[2]/div[1]/div[1]/h1').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[6]/div/div[2]/div[1]/div[1]').text)
            except:
                info.append("")
        #상장일자
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/div/div[2]/div[1]/table/tbody/tr[5]/td[2]').text + " JPY")
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/div/div[2]/div[1]/table/tbody/tr[6]/td[2]').text + " JPY")
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        #거래량
        info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/div/div[2]/div[1]/table/tbody/tr[7]/td[2]/fin-streamer').text)
        #시가총액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/div/div[2]/div[2]/table/tbody/tr[1]/td[2]').text)
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Japan")
        info.append("Japan, Asia")
        info.append("")
        info.append("")

        # Event
        info.append("Not Available")

        # Currency Information
        info.append("JPY")
        info.append("Yen")

        # Management
        info.append("Chris")
        info.append("Yahoo Finance")
        info.append("2022-06-21")

        # Status Management
        print(info)
        writing.writerow(info)
    except:
        try:
            info = []
            name = df['현지언어기업명'].iloc[i]
            #국가코드
            info.append("JPN")
            info.append("Nihon")
            info.append("Japan")
            info.append("UTC+09:00")
            info.append("아시아")
            info.append("6100000000000 USD")
            info.append("125502000")
            info.append("동북아시아")
            info.append("JPN202206" + name)
            info.append(name)
            info.append(name)
            address1 = "Japan"
            address2 = "Japan, Asia"
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group). Located at " + address2 + ", they are operated as public company. You can find more information on company's website.")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group). Located at " + address2 + ", they are operated as public company. You can find more information on company's website.")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append(address1)
            info.append(address1)
            info.append(address2)
            info.append(address2)
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append(df['기업상태'].iloc[i]) 
            info.append("")
            info.append("")
            info.append("Contact")
            info.append("Contact")
            info.append("Contact")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("TTM")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("General Business")
            info.append("General Business")
            info.append("Product and Services")
            info.append("Product and Services")
            info.append("JPN")
            info.append("Nihongo")
            info.append("Japanese")
            info.append("JPX")
            info.append("Japan Exchange Group")
            info.append("Japan Exchange Group")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Japan")
            info.append("Japan, Asia")
            info.append("")
            info.append("")
            info.append("Not Available")
            info.append("JPY")
            info.append("Yen")
            info.append("Chris")
            info.append("Yahoo Finance")
            info.append("2022-06-21")

            writing.writerow(info)
        except:
            failList.append(df['현지언어기업명'].iloc[i])
            
# Check failed result 
# - There are many ways to deal with failList: retry, ignore, manually add, find why error has occured.
# - This will be up to user's choice.
print("failLength:")
print(len(failList))
print(failList)

In [ ]:
df = pd.read_excel (r'C:\Users\user\Desktop\Japan\Japan Company List - Completed.xlsx')
path = r'C:\Users\user\Desktop\자료\chromedriver'
cover = open('Japan2019 condensed.csv','w', newline='', encoding='UTF8')
writing = csv.writer(cover)
writing.writerow(['영문기업명','영문한줄소개내용','영문기업소개내용','기업홈페이지URL','영문기업주소','영문기업상세주소','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','영문산업군명','영문주요제품명내용'])
# Basic Setting
failList = []
driver = webdriver.Chrome(path)
driver.get('https://finance.yahoo.com/quote/1301.T/holders?p=1301.T')
time.sleep(3)
# Range must be changed
for i in range (0,3819):
    try:
        # Search Setting
        name = df['현지언어기업명'].iloc[i]
        info = []
        # Status Check
        print(i)
        print(name)

        # Enter name of company in searchbox, and wait for 2 seconds.
        try:
            driver.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div[1]/div/div[2]/div/form/input[1]").send_keys(df['현지언어기업명'].iloc[i])
        except:
            driver.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div/div/div[1]/div/div[2]/div/form/input[1]").send_keys(df['현지언어기업명'].iloc[i])
        time.sleep(2)
        # Get to the company information page
        try:
            driver.find_element(By.XPATH, value='//*[@id="result-quotes-0"]/div[1]').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[1]/div/div/div/div/div/div/div/div/div/div/div/div/div[2]/div/div[2]/form/div/div[1]/div/ul[1]/li[1]').click()
        time.sleep(2)
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/div/div/section/div/ul/li[6]/a').click()
        time.sleep(2)
        
        #Company Identification
        info.append(name)

        #Descripton, Contact, Address, Extra Information, Management Information
        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan, Asia"
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[2]').text + ", Japan, Asia"
            except:
                try:
                    address1 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[1]').text + ", Japan"
                    address2 = driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/text()[1]').text + ", Japan, Asia"
                except:
                    address1 = "Japan"
                    address2 = "Japan, Asia"
        try:
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[2]/p').text)
        except:
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on JPX(Japan Exchange Group). Located at " + address2 + ", they are providing services related to  " + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[4]').text + ". You can find more information on company's website, " + driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[2]').text)
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[1]/a[2]').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul').text) 
            except:
                info.append("")
        info.append(address1)
        info.append(address2)

        # Financial Information
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul/li[7]/a').click()
        time.sleep(2)
        #회계연도

        try:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[2]/a/div').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[2]/a/div/span').click()
        time.sleep(2)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button').click()
        except:
            time.sleep(1)
            try: 
                driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/svg').click()
            except:
                try:
                    time.sleep(1)
                    driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/svg/path').click()
                except:
                    driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[2]/button/div').click()
        time.sleep(2)

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[1]/div/div[5]/span').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[1]/div/div[5]/span').text) 
            except:
                info.append("")

        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[2]/div[1]/div[1]/div[5]/span').text  + "K Yen")
            except:
                info.append("")
        #비유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[2]/div[2]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[2]/div[2]/div[1]/div[5]/span').text  + "K Yen")
            except:
                info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[1]/div[5]/span').text  + "K Yen")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[2]/div[1]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[2]/div[1]/div[1]/div[5]/span').text  + "K Yen")
            except:
                info.append("")
        #비유동부채금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[2]/div[2]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[2]/div[2]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[5]/span').text + "K Yen")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[4]/div[1]/div[1]/div[2]/div[3]/div[1]/div[5]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[3]/div[1]/div[5]/span').text  + "K Yen")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[1]/a').click()
        time.sleep(2)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[1]/div[6]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[1]/div[6]/span').text + "K Yen")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[1]/div[6]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[1]/div[6]/span').text + "K Yen")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[6]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[2]/div[1]/div[1]/div[6]/span').text + "K Yen")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[5]/div[1]/div[6]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[5]/div[1]/div[6]/span').text + "K Yen")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[6]/div[2]/div[1]/div[1]/div[6]/span').text + "K Yen")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[6]/div[2]/div[1]/div[1]/div[6]/span').text + "K Yen")
            except:
                info.append("")
        #금융비용금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[6]/div[2]/div[2]/div[1]/div[6]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[6]/div[2]/div[2]/div[1]/div[6]/span').text + "K Yen")
            except:
                info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[7]/div[1]/div[6]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[23]/div[1]/div[6]/span').text + "K Yen")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[8]/div[1]/div[6]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[8]/div[1]/div[6]/span').text + "K Yen")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[9]/div[2]/div[1]/div[2]/div[1]/div[2]/div/div[1]/div[6]').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[27]/div[1]/div[6]/span').text + "K Yen")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[1]/div[1]/div/div[3]/a').click()
        time.sleep(2)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[1]/div[1]/div[6]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[1]/div[1]/div[6]/span').text + "K Yen")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[2]/div[1]/div[6]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[2]/div[1]/div[6]/span').text + "K Yen")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[3]/div[1]/div[6]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[3]/div[1]/div[6]/span').text + "K Yen")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[2]/div[3]/div[1]/div[6]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[2]/div[2]/div[1]/div[6]/span').text + "K Yen")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[4]/div[1]/div[1]/div[2]/div[4]/div[1]/div[6]/span').text + "K Yen")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/section/div[3]/div[1]/div/div[2]/div[4]/div[1]/div[6]/span').text + "K Yen")
            except:
                info.append("")

        try:
            # Industry
            driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[7]/section/div/ul/li[6]/a').click()
            time.sleep(2)
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[2]').text)
            except:
                info.append("General Business")
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[4]').text)
            except:
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/div[1]/div/div/p[2]/span[2]').text)
                except:
                    info.append("Product and Services")
        except:
            info.append("General Business")
            info.append("Product and Services")
        # Status Management
        print(info)
        writing.writerow(info)
    except:
        print("fail")

print("DONE")